# Create ISA-API Investigation from Datascriptor Study Design configuration

In this notebook I will show you how you can use a study design configuration is JSON format as produce by datascriptor (https://gitlab.com/datascriptor/datascriptor) to generate a single-study ISA investigation and how you can then serialise it in JSON and tabular (i.e. CSV) format.

Or study design configuration consists of:
- a 4-arm study design. Each arm has 10 subjects
- there is an observational factor, named "condition" with two values: "healthy" and "diseased"
- a crossover of two treatments, a drug treatment and a biological treatment
- three non-treatment phases: screen, washout and follow-up
- two sample types colllected: blood and derma
- two assay types: 
    - metabolite profiling through mass spectrometry on the derma sample. The mass spec will be run on a "Agilent QTQF 6510" instrument, testing both "FIA" and "LC" injection modes, and "positive" acquisition mode.
    - metabolite profiling through  NMR spectroscopy on the blood samples.  The NMR will be run on a "Bruker Avance II 1 GHz" instrument, on "1D 1H NMR" acquisition mode, testing both "CPGM" amd "TOCSY" pulse sequences.

## 1. Setup

Let's import all the required libraries

In [7]:
from time import time
import os
import json

In [8]:
## ISA-API related imports
from isatools.model import Investigation, Study

In [9]:
## ISA-API create mode related imports
from isatools.create.model import StudyDesign
from isatools.create.connectors import generate_study_design_from_config

# serializer from ISA Investigation to JSON
from isatools.isajson import ISAJSONEncoder

# ISA-Tab serialisation
from isatools import isatab

In [10]:
## ISA-API create mode related imports
from isatools.create import model
from isatools import isajson

## 2. Load the Study Design JSON configuration

First of all we load the study design configurator with all the specs defined above

In [12]:
with open(os.path.abspath(os.path.join(
    "config", "crossover-study-design-4-arms-healthy-diseased-nmr-ms.json"
)), "r") as config_file:
    study_design_config = json.load(config_file)
study_design_config

{'name': 'Crossover study with observational factors ',
 'description': 'Or study design configuration consists of:\na 4-arm (i.e. groups) study design. Each arm has 10 subjects\nthere is an observational factor , named "condition" with two values: "healthy" and "diseased"\na crossover of two treatments: a drug treatment (ibuprofen) and a biological treatment (KpJH46Φ2 phage injection)\nthree non-treatment phases: screen, washout and follow-up\ntwo sample types colllected: blood and derma\ntwo assay types: (1) metabolite profiling through mass spectrometry on the saliva sample. The mass spec will be run on a "Agilent QTQF 6510" instrument, testing both "FIA" and "LC" injection modes, and "positive" acquisition mode.(2) metabolite profiling through NMR spectroscopy on the blood samples. The NMR will be run on a "Bruker Avance II 1 GHz" instrument, on "1D 1H NMR" acquisition mode, testing both "CPGM" amd "TOCSY" pulse sequences.',
 'subjectType': 'Homo sapiens',
 'subjectSize': 10,
 'des

## 3. Generate the ISA Study Design from the JSON configuration
To perform the conversion we just need to use the function `generate_isa_study_design_from_config` (name possibly subject to change, should we drop the "isa" and "datascriptor" qualifiers?)

In [13]:
study_design = generate_study_design_from_config(study_design_config)
assert isinstance(study_design, StudyDesign)

## 4. Generate the ISA Study from the StudyDesign and embed it into an ISA Investigation

The `StudyDesign.generate_isa_study()` method returns the complete ISA-API `Study` object.

In [14]:
start = time()
study = study_design.generate_isa_study()
end = time()
print('The generation of the study design took {:.2f} s.'.format(end - start))
assert isinstance(study, Study)
investigation = Investigation(identifier='inv01', studies=[study])

The generation of the study design took 2.24 s.


## 5. Serialize and save the JSON representation of the generated ISA Investigation

In [15]:
start = time()
inv_json = json.dumps(investigation, cls=ISAJSONEncoder, sort_keys=True, indent=4, separators=(',', ': '))
end = time()
print('The JSON serialisation of the ISA investigation took {:.2f} s.'.format(end - start))

The JSON serialisation of the ISA investigation took 0.72 s.


In [16]:
directory = os.path.abspath(os.path.join('output', 'crossover-4-arms'))
if not os.path.exists(directory):
    os.makedirs(directory)
with open(os.path.abspath(os.path.join('output', 'crossover-4-arms', 'isa-investigation-crossover-4-arms.json')), 'w') as out_fp:
    json.dump(json.loads(inv_json), out_fp)

## 6. Dump the ISA Investigation to ISA-Tab

In [17]:
start = time()
isatab.dump(investigation, os.path.abspath(os.path.join('output', 'crossover-4-arms')))
end = time()
print('The Tab serialisation of the ISA investigation took {:.2f} s.'.format(end - start))

The Tab serialisation of the ISA investigation took 26.52 s.


To use them on the notebook we can also dump the tables to pandas DataFrames, using the `dump_tables_to_dataframes` function rather than dump

In [18]:
dataframes = isatab.dump_tables_to_dataframes(investigation)

In [19]:
len(dataframes)

3

## 7. Check the correctness of the ISA-Tab DataFrames 

We have 1 study file and 2 assay files (one for MS and one for NMR). Let's check the names:

In [20]:
for key in dataframes.keys():
    display(key)

's_study_01.txt'

'a_AT2_metabolite-profiling_NMR-spectroscopy.txt'

'a_AT1_metabolite-profiling_mass-spectrometry.txt'

### 7.1 Count of subjects and samples

We have 10 subjects in the each of the six arms for a total of 60 subjects. 5 blood samples per subject are collected (1 in treatment 1 phase, 1 in treatment, and 3 in the follow-up phase) for a total of 300 blood samples. These will undergo the NMR assay. We have 4 saliva samples per subject (1 during screen and 3 during follow-up) for a total of 240 saliva samples. These will undergo the "mass spcetrometry" assay.

In [21]:
study_frame = dataframes['s_study_01.txt']
count_arm0_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP0' in el)])
count_arm1_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP1' in el)])
count_arm2_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP2' in el)])
count_arm3_samples = len(study_frame[study_frame['Source Name'].apply(lambda el: 'GRP3' in el)])
print("There are {} samples in the GRP0 arm (i.e. group)".format(count_arm0_samples))
print("There are {} samples in the GRP1 arm (i.e. group)".format(count_arm1_samples))
print("There are {} samples in the GRP2 arm (i.e. group)".format(count_arm2_samples))
print("There are {} samples in the GRP3 arm (i.e. group)".format(count_arm3_samples))

There are 130 samples in the GRP0 arm (i.e. group)
There are 130 samples in the GRP1 arm (i.e. group)
There are 130 samples in the GRP2 arm (i.e. group)
There are 130 samples in the GRP3 arm (i.e. group)


In [22]:
dataframes['a_AT1_metabolite-profiling_mass-spectrometry.txt']

,Sample Name,Protocol REF,Performer,Extract Name,Characteristics[extract type],Term Accession Number,Protocol REF.1,Performer.1,Labeled Extract Name,Label,Protocol REF.2,Parameter Value[instrument],Term Accession Number.1,Parameter Value[injection_mode],Term Accession Number.2,Parameter Value[acquisition_mode],Term Accession Number.3,MS Assay Name,Performer.2,Raw Spectral Data File
0,GRP0_SBJ01_A0E0_SMP-derma-1,extraction,Unknown,AT1-S4-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S4-LE-R1,label_0,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,LC,,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S4-mass-spectrometry-Acquisition-R2,Unknown,AT1-S4-raw-spectral-data-file-R2
1,GRP0_SBJ01_A0E0_SMP-derma-1,extraction,Unknown,AT1-S4-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S4-LE-R2,label_0,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,LC,,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S4-mass-spectrometry-Acquisition-R4,Unknown,AT1-S4-raw-spectral-data-file-R4
2,GRP0_SBJ01_A0E0_SMP-derma-1,extraction,Unknown,AT1-S4-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S4-LE-R1,label_0,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,FIA,http://purl.obolibrary.org/obo/MS_1000058,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S4-mass-spectrometry-Acquisition-R1,Unknown,AT1-S4-raw-spectral-data-file-R1
3,GRP0_SBJ01_A0E0_SMP-derma-1,extraction,Unknown,AT1-S4-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S4-LE-R2,label_0,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,FIA,http://purl.obolibrary.org/obo/MS_1000058,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S4-mass-spectrometry-Acquisition-R3,Unknown,AT1-S4-raw-spectral-data-file-R3
4,GRP0_SBJ01_A0E4_SMP-derma-1,extraction,Unknown,AT1-S20-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S20-LE-R2,label_0,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,LC,,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S20-mass-spectrometry-Acquisition-R4,Unknown,AT1-S20-raw-spectral-data-file-R4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,GRP3_SBJ10_A3E4_SMP-derma-2,extraction,Unknown,AT1-S147-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S147-LE-R1,label_0,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,LC,,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S147-mass-spectrometry-Acquisition-R2,Unknown,AT1-S147-raw-spectral-data-file-R2
636,GRP3_SBJ10_A3E4_SMP-derma-3,extraction,Unknown,AT1-S148-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S148-LE-R1,label_0,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,FIA,http://purl.obolibrary.org/obo/MS_1000058,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S148-mass-spectrometry-Acquisition-R1,Unknown,AT1-S148-raw-spectral-data-file-R1
637,GRP3_SBJ10_A3E4_SMP-derma-3,extraction,Unknown,AT1-S148-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S148-LE-R2,label_0,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,FIA,http://purl.obolibrary.org/obo/MS_1000058,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S148-mass-spectrometry-Acquisition-R3,Unknown,AT1-S148-raw-spectral-data-file-R3
638,GRP3_SBJ10_A3E4_SMP-derma-3,extraction,Unknown,AT1-S148-Extract-R1,polar fraction,polar fraction,labeling,Unknown,AT1-S148-LE-R2,label_0,mass spectrometry,Agilent QTQF 6510,http://purl.obolibrary.org/obo/MS_1000676,LC,,positive mode,http://purl.obolibrary.org/obo/MS_1002807,AT1-S148-mass-spectrometry-Acquisition-R4,Unknown,AT1-S148-raw-spectral-data-file-R4


In [23]:
dataframes['a_AT2_metabolite-profiling_NMR-spectroscopy.txt']

,Sample Name,Protocol REF,Performer,Extract Name,Characteristics[extract type],Protocol REF.1,Parameter Value[instrument],Parameter Value[acquisition_mode],Parameter Value[pulse_sequence],NMR Assay Name,Performer.1,Free Induction Decay Data File
0,GRP0_SBJ01_A0E0_SMP-blood-1,extraction,Unknown,AT2-S4-Extract-R1,supernatant,NMR spectroscopy,Bruker Avance II 1 GHz,1D 1H NMR,TOCSY,AT2-S4-NMR-spectroscopy-Acquisition-R1,Unknown,AT2-S4-raw_spectral_data_file-R1
1,GRP0_SBJ01_A0E0_SMP-blood-1,extraction,Unknown,AT2-S4-Extract-R2,pellet,NMR spectroscopy,Bruker Avance II 1 GHz,1D 1H NMR,TOCSY,AT2-S4-NMR-spectroscopy-Acquisition-R8,Unknown,AT2-S4-raw_spectral_data_file-R8
2,GRP0_SBJ01_A0E0_SMP-blood-1,extraction,Unknown,AT2-S4-Extract-R2,pellet,NMR spectroscopy,Bruker Avance II 1 GHz,1D 1H NMR,CPMG,AT2-S4-NMR-spectroscopy-Acquisition-R5,Unknown,AT2-S4-raw_spectral_data_file-R5
3,GRP0_SBJ01_A0E0_SMP-blood-1,extraction,Unknown,AT2-S4-Extract-R1,supernatant,NMR spectroscopy,Bruker Avance II 1 GHz,1D 1H NMR,CPMG,AT2-S4-NMR-spectroscopy-Acquisition-R3,Unknown,AT2-S4-raw_spectral_data_file-R3
4,GRP0_SBJ01_A0E0_SMP-blood-1,extraction,Unknown,AT2-S4-Extract-R2,pellet,NMR spectroscopy,Bruker Avance II 1 GHz,1D 1H NMR,TOCSY,AT2-S4-NMR-spectroscopy-Acquisition-R7,Unknown,AT2-S4-raw_spectral_data_file-R7
...,...,...,...,...,...,...,...,...,...,...,...,...
2875,GRP3_SBJ10_A3E4_SMP-blood-6,extraction,Unknown,AT2-S336-Extract-R2,pellet,NMR spectroscopy,Bruker Avance II 1 GHz,1D 1H NMR,CPMG,AT2-S336-NMR-spectroscopy-Acquisition-R5,Unknown,AT2-S336-raw_spectral_data_file-R5
2876,GRP3_SBJ10_A3E4_SMP-blood-6,extraction,Unknown,AT2-S336-Extract-R1,supernatant,NMR spectroscopy,Bruker Avance II 1 GHz,1D 1H NMR,CPMG,AT2-S336-NMR-spectroscopy-Acquisition-R3,Unknown,AT2-S336-raw_spectral_data_file-R3
2877,GRP3_SBJ10_A3E4_SMP-blood-6,extraction,Unknown,AT2-S336-Extract-R2,pellet,NMR spectroscopy,Bruker Avance II 1 GHz,1D 1H NMR,TOCSY,AT2-S336-NMR-spectroscopy-Acquisition-R8,Unknown,AT2-S336-raw_spectral_data_file-R8
2878,GRP3_SBJ10_A3E4_SMP-blood-6,extraction,Unknown,AT2-S336-Extract-R1,supernatant,NMR spectroscopy,Bruker Avance II 1 GHz,1D 1H NMR,CPMG,AT2-S336-NMR-spectroscopy-Acquisition-R4,Unknown,AT2-S336-raw_spectral_data_file-R4


# 7.1 Overview of the Mass Spec assay table

For the mass. spec. assay table, we have 160 (derma) samples, 160 extracts (1 per  sample, "polar" fraction), 320 labeled extracts (2 per extract, as "#replicates" is  2) and 640 mass spec protocols + 640 output files (2 per labeled extract as we do 1 technical replicate with 2 protocol parameter combinations `["Agilent QTQF 6510", "FIA", "positive mode"]` and `["Agilent QTQF 6510", "LC", "positive mode"]`).

In [24]:
dataframes['a_AT1_metabolite-profiling_mass-spectrometry.txt'].nunique(axis=0, dropna=True)

Sample Name                          160
Protocol REF                           1
Performer                              1
Extract Name                         160
Characteristics[extract type]          1
Term Accession Number                  1
Protocol REF.1                         1
Performer.1                            1
Labeled Extract Name                 320
Label                                  1
Protocol REF.2                         1
Parameter Value[instrument]            1
Term Accession Number.1                1
Parameter Value[injection_mode]        2
Term Accession Number.2                2
Parameter Value[acquisition_mode]      1
Term Accession Number.3                1
MS Assay Name                        640
Performer.2                            1
Raw Spectral Data File               640
dtype: int64

### Overview of the NMR assay table

For the NMR assay table, we have 360 (blood) samples, 720 extracts (2 per  sample, a single replicate of the "supernatant" and "pellet" fractions) and 2880 NMR protocols + 4800 output files (4 per extract as we do 2 technical replicates with 2 protocol parameter combinations `["Bruker Avance II 1 GHz", "1D 1H NMR", "CPGM"]` and `["Bruker Avance II 1 GHz", "1D 1H NMR", "TOCSY"]`).

In [25]:
dataframes['a_AT2_metabolite-profiling_NMR-spectroscopy.txt'].nunique(axis=0, dropna=True)

Sample Name                           360
Protocol REF                            1
Performer                               1
Extract Name                          720
Characteristics[extract type]           2
Protocol REF.1                          1
Parameter Value[instrument]             1
Parameter Value[acquisition_mode]       1
Parameter Value[pulse_sequence]         2
NMR Assay Name                       2880
Performer.1                             1
Free Induction Decay Data File       2880
dtype: int64